<a href="https://colab.research.google.com/github/janiosl/python.ds/blob/master/ml/PyTorch/ML_model_reuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
#Conecta o Google Colab ao Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Define o path dos códigos e dados como diretório de trabalho atual
###ALTERAR PARA O DIRETÓRIO DO SEU GOOGLE DRIVE OU LOCAL ALTERNATIVO
import os
dir = '/content/drive/My Drive/Mestrado PPCIC/03. Curso/3. Aprendizado/t3'
os.chdir(dir)

In [4]:
##============================================
##Preparação do Ambiente
##============================================

#Bibliotecas para carga e tratamento
import pandas as pd
import numpy as np


#Importações do PyTorch para criação de modelos e preparação de variáveis
import torch
from torch.autograd import Variable


#Biblioteca para cálculo das métricas de avaliação e tratamento dos datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#Importações do PyTorch para criação da Rede Neural
import torch.nn as nn
import torch.nn.functional as F

#Biblioteca para carga do modelo treinado
import pickle

In [5]:
#Definção dos títulos das colunas
headers = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC',
           'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']

In [6]:
#Carregar conjunto de dados para análise
#Carga do conjunto de teste
arquivo = 'https://raw.githubusercontent.com/MLRG-CEFET-RJ/ml-class/master/ml-t3/datasets/credtest.txt'
data_test = pd.read_csv(arquivo, sep='\t', header=None, names=headers)

In [7]:
#Usando a lógica estuda para recarregar os dados e refazer a separação
data_test_ = pd.get_dummies(data=data_test,
               prefix='ESCT',
               columns=['ESCT'],
               drop_first=True)

data_test_.head()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,2,500,1,618,10,85,0,36,6,0,0,0,0,0
1,0,813,0,552,4,119,0,43,48,119,1,1,0,0
2,0,350,0,488,12,66,0,43,0,0,1,0,0,1
3,0,1530,0,381,1,398,0,28,48,0,1,1,0,0
4,0,688,1,396,10,60,0,49,72,0,1,0,0,0


In [8]:
#Transformação dos atributos e da classe alvo em matrizes
y_test = np.array(data_test['CLASSE'])

In [9]:
#Separação dos atributos precisa atentar que a ordem das colunas mudou
##=============================================================================

#Recuperação dos nomes das colunas
atributos = list(data_test_.columns)

#Eliminação da colunas com a classe na lista de atributos
atributos.remove('CLASSE')

In [10]:
#Transformação dos atributos e da classe alvo em matrizes
X_test = np.array(data_test_.loc[:, atributos])

In [11]:
#Criação do transformador para noramlização
norm = StandardScaler()

#Normalização das características
X_test = norm.fit_transform(X_test)

In [12]:
#Atributos e classes de teste
X_test = Variable(torch.Tensor(X_test).float())
y_test = Variable(torch.Tensor(y_test).long())

In [13]:
#Instalação da biblioteca para integração da sintonização no Pythorch
!pip install skorch

#Biblioteca para adaptação sintonização de Hiperparâmetro ao Pytorch
from skorch import NeuralNetClassifier
#Biblioteca para sintonização de Hiperparâmetro
from sklearn.model_selection import GridSearchCV

In [14]:
#Apontar para sua rede neural
#O pickle carregar o modelo treinado, mas a classe original é necessária
#Exemplo disponível em:
# https://github.com/janiosl/python.ds/blob/master/ml/PyTorch/ML_PyTorch_MLP.py
from rna.ML_PyTorch_MLP import Net_

In [15]:
#Carregar modelo armazenado
with open('target/JANIO_T3_MLP_GS.pkl', 'rb') as f:
    model = pickle.load(f)


print('Modelo salvo\nPara usar, basta reativar o código acima')

Modelo salvo
Para usar, basta reativar o código acima


In [16]:
#Aplicar o modelo carregado para realizar predições

#Predição no conjunto de testes
y_pred = model.predict(X_test)

#Avaliação Final da Rede Neural nos dados de teste

#Matriz de confusão
print('Matriz de confusão')
print(confusion_matrix(y_test.data, y_pred))

#Relatório de avaliação da classificação
print('-' * 60, end='\n\n')
print('Relatório de avaliação da classificação')
print(classification_report(y_test.data, y_pred,
                            zero_division=0))

Matriz de confusão
[[301   5]
 [ 49 222]]
------------------------------------------------------------

Relatório de avaliação da classificação
              precision    recall  f1-score   support

           0       0.86      0.98      0.92       306
           1       0.98      0.82      0.89       271

    accuracy                           0.91       577
   macro avg       0.92      0.90      0.90       577
weighted avg       0.92      0.91      0.91       577

